In [ ]:
import pandas as pd
from src.Get_data_DB import DataTransformer
from rapidfuzz import fuzz, process

transform=DataTransformer()

Product_query='select Ma_khoa_hoc, Ten_khoa_hoc from Dim_Khoa_hoc where Ma_khoa_hoc != 550'
Class_query='select Ma_lop_hoc, Ten_lop_hoc from Dim_Lop_hoc'

df_product=transform.fetch_from_sql_server(Product_query)
df_class=transform.fetch_from_sql_server(Class_query)



df1=pd.read_csv("~/DWH_Cole_Project/data_tmp/Ke_hoach_Sale_TOA.csv")
df2=pd.read_csv("~/DWH_Cole_Project/data_tmp/Ke_hoach_Sale_TOT.csv")

In [ ]:
# Các hàm để đảm bảo tên khoá học chuẩn trong file kế hoạch
def build_standard_list(series):
    """Tạo danh sách chuẩn từ Series: loại bỏ trùng và chuẩn hoá chữ thường"""
    clean_series = (
        series.dropna()
        .drop_duplicates()
        .astype(str)
        .str.strip()
    )
    standard_list = clean_series.str.lower().tolist()
    standard_map = dict(zip(clean_series.str.lower(), clean_series))
    return standard_list, standard_map

def match_course_name(value, standard_list, standard_map, threshold=60):
    """Tìm match fuzzy cho 1 tên khoá học"""
    if pd.isna(value):
        return "Khác"

    val = str(value).strip().lower()
    result = process.extractOne(val, standard_list, scorer=fuzz.ratio)

    if result is None:
        return "Khác"

    match, score, _ = result
    if score >= threshold:
        return standard_map[match]
    else:
        return "Khác"

def standardize_course_column(input_series, standard_list, standard_map, threshold=60):
    """Áp dụng chuẩn hoá cho cả cột"""
    return input_series.apply(lambda x: match_course_name(x, standard_list, standard_map, threshold))

In [ ]:
df1=df1.rename(columns={
    "Dữ liệu kế hoạch theo tháng":"Nam",
    "Unnamed: 1":"Thang",
    "Unnamed: 2": "Ten_khoa_hoc",
    "Unnamed: 3": "Thu_tu_lop",
    "Unnamed: 4":"L1",
    "Unnamed: 5": "L1_L1/Sale",
    "Unnamed: 6": "L1_L1C",
    "Unnamed: 7": "L2",
    "Unnamed: 8":"Khong xac dinh",
    "Unnamed: 9": "L7",
    "Unnamed: 10":"L8",
    "Unnamed: 11":"L7+L8",
    "Unnamed: 12":"Tuan 1",
    "Unnamed: 13":"Tuan 2",
    "Unnamed: 14":"Tuan 3",
    "Unnamed: 15":"Tuan 4",
    "Unnamed: 16":"Tuan 5",
    "Unnamed: 17":"Doanh_so",
    "Unnamed: 18":"Doanh_so/Sale",
})
df1=df1.drop(columns=["L7+L8","Khong xac dinh","L1_L1C"])

In [ ]:
df1['Nam']=df1['Nam'].ffill()
df1['Thang']=df1['Thang'].ffill()
df1['Ten_khoa_hoc']=df1['Ten_khoa_hoc'].ffill()
df1=df1.fillna(0)

In [ ]:
df1['Thang'] = df1['Thang'].str.extract(r'(\d+)')[0].astype(int)

In [ ]:
df1['Ten_khoa_hoc'] = df1['Ten_khoa_hoc'].str.replace('\n', '', regex=False)

In [ ]:
df1['Ten_khoa_hoc'] = df1['Ten_khoa_hoc'].replace(['Khóa DS, DA ','Khóa DS, DA'], 'DS.01')

In [ ]:
# In hoa cột
df1['Thu_tu_lop'] = df1['Thu_tu_lop'].str.upper()

# Trích xuất mã K
df1['Thu_tu_lop'] = df1['Thu_tu_lop'].str.extract(r'(K\d+)')[0]


In [ ]:
df1 = df1.dropna(subset=['Thu_tu_lop'])

In [ ]:
# Tìm mã K có độ dài = 2 và thêm số 0
mask = (df1['Thu_tu_lop'].str.len() == 2) & df1['Thu_tu_lop'].notna()
df1.loc[mask, 'Thu_tu_lop'] = df1.loc[mask, 'Thu_tu_lop'].str.replace(r'K(\d)', r'K0\1', regex=True)

In [ ]:
# Bước 1: Tạo danh sách chuẩn
standard_list, standard_map = build_standard_list(df_product['Ten_khoa_hoc'].drop_duplicates())

# Bước 2: Chuẩn hoá cột df_cf['Ten_khoa_hoc']
df1['Ten_khoa_hoc'] = standardize_course_column(
    df1['Ten_khoa_hoc'],
    standard_list,
    standard_map,
    threshold=75
)

In [ ]:
df1['Ten_lop_hoc'] = df1['Ten_khoa_hoc'].astype(str) + '-' + df1['Thu_tu_lop'].astype(str)
df1=df1.drop(columns=['Ten_khoa_hoc','Thu_tu_lop'])

In [ ]:
df1=df1.merge(df_class,on='Ten_lop_hoc', how='inner')

In [ ]:
pattern = r'[.đ\s]'  # Loại bỏ dấu chấm, chữ đ và khoảng trắng
df1['Doanh_so'] = df1['Doanh_so'].str.replace(pattern, '', regex=True)
df1['Doanh_so/Sale'] = df1['Doanh_so/Sale'].str.replace(pattern, '', regex=True)
df1=df1.drop(columns='Ten_lop_hoc')

In [ ]:
df1

In [ ]:
# Dữ liệu TOA theo Tháng
cols_thang = ['Nam', 'Thang', 'L1', 'L1_L1/Sale', 'L2','L7', 'L8', 'Doanh_so', 'Doanh_so/Sale', 'Ma_lop_hoc']
df_thang = df1[cols_thang].copy()

----

In [37]:
import pandas as pd
import os
from rapidfuzz import fuzz, process
from src.Get_data_DB import DataTransformer
from src.Process_utm import ColumnStandardizer


standardizer = ColumnStandardizer(
    threshold=75,
    preserve_if_low_similarity=[]
)
transformer=DataTransformer()

# Các hàm để chuẩn hoá tên khoá học trong tên chiến dịch
def build_standard_list(series):
    """Tạo danh sách chuẩn từ Series: loại bỏ trùng và chuẩn hoá chữ thường"""
    clean_series = (
        series.dropna()
        .drop_duplicates()
        .astype(str)
        .str.strip()
    )
    standard_list = clean_series.str.lower().tolist()
    standard_map = dict(zip(clean_series.str.lower(), clean_series))
    return standard_list, standard_map

def match_course_name(value, standard_list, standard_map, threshold=60):
    """Tìm match fuzzy cho 1 tên khoá học"""
    if pd.isna(value):
        return "Khác"

    val = str(value).strip().lower()
    result = process.extractOne(val, standard_list, scorer=fuzz.ratio)

    if result is None:
        return "Khác"

    match, score, _ = result
    if score >= threshold:
        return standard_map[match]
    else:
        return "Khác"

def standardize_course_column(input_series, standard_list, standard_map, threshold=60):
    """Áp dụng chuẩn hoá cho cả cột"""
    return input_series.apply(lambda x: match_course_name(x, standard_list, standard_map, threshold))

Query_KH=""" select Ma_khoa_hoc, Ten_khoa_hoc 
            from Dim_Khoa_hoc """

# Đọc dữ liệu
df_KH=transformer.fetch_from_sql_server(Query_KH)

df_L78=pd.read_csv("~/DWH_Cole_Project/data_tmp/L78_FA_Marketing.csv")
# Định nghĩa cấu trúc DataFrame mẫu khi file rỗng
empty_df_template = pd.DataFrame(columns=['Campaign ID', 'Campaign Name', 'Date', 'Spend'])

# Hàm kiểm tra và đọc file CSV
def read_csv_safe(file_path):
    full_path = os.path.expanduser(file_path)
    
    # Kiểm tra file có tồn tại và có kích thước > 0 byte không
    if not os.path.exists(full_path) or os.path.getsize(full_path) == 0:
        print(f"File {file_path} rỗng hoặc không tồn tại. Tạo DataFrame rỗng.")
        return empty_df_template.copy()
    
    try:
        # Thử đọc file CSV
        df = pd.read_csv(full_path)
        
        # Kiểm tra nếu DataFrame đọc được có dữ liệu
        if df.empty:
            print(f"File {file_path} không có dữ liệu. Tạo DataFrame rỗng.")
            return empty_df_template.copy()
            
        return df
    
    except pd.errors.EmptyDataError:
        print(f"File {file_path} không có dữ liệu (EmptyDataError). Tạo DataFrame rỗng.")
        return empty_df_template.copy()
    except Exception as e:
        print(f"Lỗi khi đọc file {file_path}: {str(e)}. Tạo DataFrame rỗng.")
        return empty_df_template.copy()

# Đường dẫn file

file_path3 = "~/DWH_Cole_Project/data_tmp/spend_C9_PAUSED.csv"   #Lấy dữ liệu 1 lần duy nhất ở lần chạy đầu tiên
file_path4 = "~/DWH_Cole_Project/data_tmp/spend_Cole8_PAUSED.csv"
file_path1 = "~/DWH_Cole_Project/data_tmp/spend_C9_ACTIVE.csv"
file_path2 = "~/DWH_Cole_Project/data_tmp/spend_Cole8_ACTIVE.csv"

# Đọc dữ liệu
df_cf1 = read_csv_safe(file_path1)
df_cf2 = read_csv_safe(file_path2)
df_cf3=pd.read_csv(file_path3)
df_cf4=pd.read_csv(file_path4)
df_cf = pd.concat([df_cf1,df_cf2,df_cf3,df_cf4], ignore_index=True)

df_cf['Ten_khoa_hoc'] = df_cf['Campaign Name'].str.split('_').str[1]
df_cf['Ma_marketer'] = df_cf['Campaign Name'].str.split('_').str[2]

df_cf['Ma_marketer'] = standardizer.transform(df_cf['Ma_marketer'])


# Bước 1: Tạo danh sách chuẩn
standard_list, standard_map = build_standard_list(df_KH['Ten_khoa_hoc'].drop_duplicates())

# Trường hợp đặc biệt 6 tháng đầu năm 2025, Ai.Nocode chạy cho khoá DTDN
df_cf['Date'] = pd.to_datetime(df_cf['Date'])
mask = (df_cf['Ten_khoa_hoc'] == 'Ai.Nocode') & (df_cf['Date'] >= '2025-01-01') & (df_cf['Date'] <= '2025-10-01')
df_cf.loc[mask, 'Ten_khoa_hoc'] = 'DTDN'   

df_cf['Ten_khoa_hoc'] = df_cf['Ten_khoa_hoc'].replace('BI', 'BI.01', regex=False)

# Bước 2: Chuẩn hoá cột df_cf['Ten_khoa_hoc']
df_cf['Ten_khoa_hoc'] = standardize_course_column(
    df_cf['Ten_khoa_hoc'],
    standard_list,
    standard_map,
    threshold=70
)

df_cf["Date"] = pd.to_datetime(df_cf["Date"]).dt.date
df_L78["Thoi_gian"] = pd.to_datetime(df_L78["Thoi_gian"]).dt.date
df_cf = df_cf.rename(columns={
    'Spend': 'Chi_phi',
    'Date': 'Thoi_gian'
})
df_cf=df_cf.merge(df_KH,on='Ten_khoa_hoc',how='inner')
df_cf = df_cf.drop(columns=['Campaign ID', 'Campaign Name','Ten_khoa_hoc'])


# Lấy những bản ghi thoả mãn và bản ghi có chi phí nhưng không có số chuyển đổi --> Đổ vào bảng Chi_phi_FA
df_cf = df_cf.groupby(['Thoi_gian', 'Ma_khoa_hoc','Ma_marketer'])[['Chi_phi']].sum().reset_index()

df_FA=df_cf.merge(df_L78,on=['Thoi_gian','Ma_khoa_hoc'], how='left')

# Vì sản phẩm DE.COMBO01 có 2 mã khoá nên ở bước ánh xạ từ Ten_khoa_hoc thành Ma_khoa_hoc phát sinh thành 2 bản ghi Chi phí =
# 1. Tách dữ liệu cần xử lý và phần còn lại
df_to_process = df_FA[df_FA['Ma_khoa_hoc'].isin([515, 550])]
df_others = df_FA[~df_FA['Ma_khoa_hoc'].isin([515, 550])]

# 2. Nếu có bản ghi cần xử lý
if not df_to_process.empty:
    def xu_ly_nhom(gr):
        if len(gr) == 2:
            na_mask = gr['L7'].isna()
            if na_mask.sum() == 1:
                return gr[~na_mask]
            elif na_mask.sum() == 2:
                if (gr['Ma_khoa_hoc'] == 515).any():
                    return gr[gr['Ma_khoa_hoc'] == 515]
                else:
                    return pd.DataFrame()
            else:
                return gr
        else:
            return gr[~gr['L7'].isna()] if gr['L7'].isna().any() else gr

    df_processed = (
        df_to_process
        .groupby(['Chi_phi', 'Thoi_gian'], group_keys=False)
        .apply(xu_ly_nhom)
    )
else:
    # Không có gì để xử lý
    df_processed = pd.DataFrame(columns=df_FA.columns)

# 3. Ghép lại dữ liệu
df_FA = pd.concat([df_processed, df_others], ignore_index=True)
df_FA = df_FA.fillna(0)
df_FA=df_FA.drop(columns='Chi_phi')

/tmp/ipykernel_15750/3116744903.py:163: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(xu_ly_nhom)


In [39]:
# Trường hợp có số L7 chuyển đổi  nhưng không có chi phí thì chuyển thành số L7_mess ở bảng Chi_phi_mess 
df_mess_bonus=df_cf.merge(df_L78, on=['Ma_khoa_hoc', 'Thoi_gian'], how='right')
df_mess_bonus=df_mess_bonus[['Thoi_gian','Chi_phi','Ma_khoa_hoc','Ma_marketer','L7','L8']]
df_mess_bonus = df_mess_bonus[df_mess_bonus['Chi_phi'].isna()]
df_mess_bonus.drop(columns='Chi_phi')


# Ghi dữ liệu vào folder data_result
df_FA.to_csv("~/DWH_Cole_Project/data_result/L78_FA_transformed.csv", index=False)
df_mess_bonus.to_csv("~/DWH_Cole_Project/data_result/L78_mess_bonus_transformed.csv", index=False)

---

In [50]:
import pandas as pd

df1=pd.read_csv("~/DWH_Cole_Project/data_tmp/So_L1C_Mess_TOT.csv")
df2=pd.read_csv("~/DWH_Cole_Project/data_tmp/So_L1_Mess_TOT.csv")
df3=pd.read_csv("~/DWH_Cole_Project/data_tmp/So_L7_Mess_TOT.csv")
df4=pd.read_csv("~/DWH_Cole_Project/data_tmp/So_L8_Mess_TOT.csv")

df_bonus=pd.read_csv("~/DWH_Cole_Project/data_result/Chi_phi_mess_bonus_transformed.csv")


# Chuẩn hoá dữ liệu 
merged=df1.merge(df2,on=['Thoi_gian','Ma_khoa_hoc'], how="outer")
merged=merged.merge(df3,on=['Thoi_gian','Ma_khoa_hoc'], how="outer")
merged=merged.merge(df4,on=['Thoi_gian','Ma_khoa_hoc'], how="outer")


In [51]:
merged

,Thoi_gian,Ma_khoa_hoc,L1C,L1,L7,L8
0,2025-01-01,515,1.0,2.0,NaN,NaN
1,2025-01-01,517,1.0,4.0,NaN,NaN
2,2025-01-01,529,1.0,1.0,NaN,NaN
3,2025-01-01,548,3.0,3.0,NaN,NaN
4,2025-01-02,515,NaN,3.0,NaN,NaN
...,...,...,...,...,...,...
1059,2025-07-30,548,NaN,5.0,1.0,1.0
1060,2025-07-30,554,1.0,1.0,NaN,NaN
1061,2025-07-31,517,1.0,1.0,NaN,NaN
1062,2025-07-31,548,2.0,4.0,NaN,NaN


In [52]:
# 1. Thay thế giá trị trống (NaN) trong cột 'L1C' bằng 0
# inplace=True sẽ thay đổi trực tiếp trên DataFrame 'merged' mà không cần gán lại
merged['L1C'] = merged['L1C'].fillna(0)

# 2. Tạo cột mới 'L1_L1C' bằng L1 - L1C
# Đảm bảo rằng cả L1 và L1C đều là kiểu số trước khi thực hiện phép trừ
# Nếu chúng có thể là kiểu object (chuỗi), bạn cần chuyển đổi chúng sang số trước.
merged['L1'] = pd.to_numeric(merged['L1'], errors='coerce')
merged['L1C'] = pd.to_numeric(merged['L1C'], errors='coerce')

merged['L1_L1C'] = merged['L1'] - merged['L1C']

merged=merged.drop(columns=['L1C'])

#----Cộng thêm dữ liệu L1 bonus 
df_bonus=df_bonus[['Thoi_gian','Ma_khoa_hoc','L1','L1_L1C','L7','L8']]
df_combined = pd.concat([df_bonus, merged], ignore_index=True)
df_result = df_combined.groupby(['Thoi_gian', 'Ma_khoa_hoc'])[['L1', 'L1_L1C','L7','L8']].sum().reset_index()
df_result = df_result[df_result['Thoi_gian'] >= '2025-01-01']
df_result.to_csv("~/DWH_Cole_Project/data_result/Count_L_of_Mess.csv",index=False)

In [54]:
df_result.to_excel("/tmp/Checks.xlsx")

---

In [1]:
import pandas as pd
from src.Get_data_DB import DataTransformer
transformer = DataTransformer()

Offline_class_query = """ select Ma_lop_hoc,
                                 Ngay_khai_giang,
                                 So_buoi_tuyen_sinh
                          from Dim_Lop_hoc
"""
# Có 2 cách tiếp cận phân nhánh lớp thuộc khoá học. Cách 1: Lớp thuộc nhiều sản phẩm ( sản phẩm lẻ và sản phẩm combo). 
# Cách 2: Lớp thuộc về 1 sản phẩm suy nhất . Trong Dim đang tổ chức theo cách hai nhưng từ Tên khoá học của Chiến dịch Meta muốn 
# về lớp thì phải dùng Cách 1
Mapping_class_query="""select oc.id Ma_lop_hoc,
                               p.id Ma_khoa_hoc
                        from offline_classes oc
                        join product_items pi on pi.item_id =oc.item_id
                        join products p on pi.product_id =p.id"""
df_LH = transformer.fetch_from_sql_server(Offline_class_query)
df_mapping=transformer.fetch_from_mysql(Mapping_class_query)
df= pd.read_csv("~/DWH_Cole_Project/data_result/Chi_phi_FA_transformed.csv")
df_78=pd.read_csv("~/DWH_Cole_Project/data_result/L78_FA_transformed.csv")

# Gộp lại dữ liệu L7, L8 vào dữ liệu gốc
df=df.merge(df_78, on=['Thoi_gian','Ma_khoa_hoc','Ma_marketer'],how='outer')
df=df.fillna(0)

In [ ]:
df.to_excel('/tmp/Check1.xlsx',

----

In [ ]:
import pandas as pd
from src.Get_data_DB import DataTransformer
transformer = DataTransformer()

Offline_class_query = """ select Ma_lop_hoc,
                                 Ngay_khai_giang,
                                 So_buoi_tuyen_sinh
                          from Dim_Lop_hoc
"""
# Có 2 cách tiếp cận phân nhánh lớp thuộc khoá học. Cách 1: Lớp thuộc nhiều sản phẩm ( sản phẩm lẻ và sản phẩm combo). 
# Cách 2: Lớp thuộc về 1 sản phẩm suy nhất . Trong Dim đang tổ chức theo cách hai nhưng từ Tên khoá học của Chiến dịch Meta muốn 
# về lớp thì phải dùng Cách 1
Mapping_class_query="""select oc.id Ma_lop_hoc,
                               p.id Ma_khoa_hoc
                        from offline_classes oc
                        join product_items pi on pi.item_id =oc.item_id
                        join products p on pi.product_id =p.id"""
df_LH = transformer.fetch_from_sql_server(Offline_class_query)
df_mapping=transformer.fetch_from_mysql(Mapping_class_query)
df= pd.read_csv("~/DWH_Cole_Project/data_result/Chi_phi_FA_transformed.csv")
df_78=pd.read_csv("~/DWH_Cole_Project/data_result/L78_FA_transformed.csv")

# Gộp lại dữ liệu L7, L8 vào dữ liệu gốc
df=df.merge(df_78, on=['Thoi_gian','Ma_khoa_hoc','Ma_marketer'],how='outer')

# Join để bảng ánh xa Lớp thuộc những sản phẩm nào ( 1 Lớp sẽ thuộc nhiều sản phẩm, đặc biệt sản phẩm bán có COMBO)
df_LH=df_LH.merge(df_mapping, on='Ma_lop_hoc', how='inner')
df['Thoi_gian'] = pd.to_datetime(df['Thoi_gian'])
df_LH['Ngay_khai_giang'] = pd.to_datetime(df_LH['Ngay_khai_giang'])

# Join dữ liệu để Chi phí về lớp
merged = df.merge(df_LH,on='Ma_khoa_hoc', how='inner')

# Điều kiện 1: So_buoi_tuyen_sinh là 0 hoặc 1 và Thoi_gian <= Ngay_khai_giang
# Đây là trường hợp sản phẩm đó mới chỉ mở 1 lớp hoặc nó thuộc lớp đầu tiên của sản phẩm đó
cond1 = (merged["So_buoi_tuyen_sinh"].isin([0, 1])) & (merged["Thoi_gian"] <= merged["Ngay_khai_giang"])

# Điều kiện 2: So_buoi_tuyen_sinh khác -1 và Thoi_gian nằm trong khoảng (Ngay_khai_giang - So_buoi_tuyen_sinh, Ngay_khai_giang)
# Dữ liệu CHi phí được lấy từ 2014, từ lúc này Ngày khai giảng của các lớp đều được điền đầy đủ
cond2 = (~merged["So_buoi_tuyen_sinh"].isin([-1])) & (
    merged["Thoi_gian"] > (merged["Ngay_khai_giang"] - pd.to_timedelta(merged["So_buoi_tuyen_sinh"], unit="D"))
) & (merged["Thoi_gian"] <= merged["Ngay_khai_giang"])

# Lọc các bản ghi thỏa mãn điều kiện
valid_records = merged[cond1 | cond2]

# Xử lí riêng cho trường hợp Chi phí chạy cho sản phẩm COMBO, khi này 1 chi phí bị lặp lại cho nhiều lớp thuộc sản phẩm đó
# Bước 1: Nhóm dữ liệu theo Thoi_gian và Ma_khoa_hoc
grouped = valid_records.groupby(['Thoi_gian', 'Ma_khoa_hoc'])


In [8]:
# Bước 2: Tạo hàm xử lý cho mỗi nhóm
def adjust_group_values(group):
    count = len(group)
    if count > 1:
        # Lấy giá trị đầu tiên của mỗi cột
        first_values = {
            'Chi_phi': group['Chi_phi'].iloc[0],
            'L1': group['L1'].iloc[0],
            'L1_L1C': group['L1_L1C'].iloc[0],
            'L7': group['L7'].iloc[0],
            'L8': group['L8'].iloc[0]
        }
        
        # Tính giá trị mới cho mỗi cột và làm tròn
        adjusted_values = {
            'Chi_phi': round(first_values['Chi_phi'] / count),
            'L1': round(first_values['L1'] / count),
            'L1_L1C': round(first_values['L1_L1C'] / count),
            'L7': round(first_values['L7'] / count),
            'L8': round(first_values['L8'] / count)
        }
        
        # Gán giá trị mới cho tất cả các bản ghi trong nhóm
        group['Chi_phi'] = adjusted_values['Chi_phi']
        group['L1'] = adjusted_values['L1']
        group['L1_L1C'] = adjusted_values['L1_L1C']
        group['L7'] = adjusted_values['L7']
        group['L8'] = adjusted_values['L8']
    
    return group

# Bước 3: Áp dụng hàm xử lý cho từng nhóm
valid_records_processed = grouped.apply(adjust_group_values)



/tmp/ipykernel_20364/1565446434.py:33: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  valid_records_processed = grouped.apply(adjust_group_values)


In [ ]:
# Bước 4: Reset index nếu cần (do groupby tạo multi-index)
valid_records_processed = valid_records_processed.reset_index(drop=True)

# Lấy các cột cần thiết
final_L78=valid_records_processed[['Thoi_gian','Ma_lop_hoc','Ma_marketer','L7','L8']]
final=valid_records_processed[['Chi_phi','Thoi_gian','Ma_lop_hoc','Ma_marketer','L1','L1_L1C']]

final.to_csv("~/DWH_Cole_Project/data_result/Chi_phi_FA_TOA_transformed.csv",index=False)
final_L78.to_csv("~/DWH_Cole_Project/data_result/L78_FA_TOA_transformed.csv",index=False)